In [1]:
import numpy as np
import os
import sys
from sentence_transformers import SentenceTransformer
import importlib.util

sys.path.append('/Users/vijaykatta/Documents/Projects/filetalk')
sys.path.append('/Users/vijaykatta/Documents/Projects/filetalk/src')
from utils.file_system_utils import FolderNode, build_tree, print_tree, fetch_parents, embedding_model

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'utils'

In [2]:
# Specify the root directory you want to analyze
root_directory = '/Users/vijaykatta/Documents/Projects/filetalk'
#root_directory = '/Users/vijaykatta/Documents'
# Build the tree
tree_root = build_tree(root_directory)

# Print the tree structure
print_tree(tree_root)

# Example usage: Fetch and print parents of a given node
# Assuming you have a reference to a node `some_node`
some_node = tree_root.children[-1]  # Example: Get the first child of root for demonstration
parents = fetch_parents(some_node)
print("\nParents from the given node to the root:")
for parent in reversed(parents):  # Reverse to print from root to the given node
    print(f'Name: {parent.name}, Path: {parent.path}')

[D] filetalk
  [D] Documentation
    [F] High level Design.drawio
    [F] process documentation.MD
  [F] .DS_Store
  [F] LICENSE
  [D] tests
    [D] test_folder
      [D] Statistics
        [F] .DS_Store
        [D] Bayesian Statistics
          [F] readme.md
        [D] ML related Stats
      [F] .DS_Store
      [D] Projects
        [F] ReaDme.md
        [D] Recommender
          [F] readme.md
        [D] Fraud Analytics
          [F] readme.MD
      [F] readme.md
    [F] usual questions.md
    [F] .DS_Store
    [D] test_utils
      [F] readme.md
    [D] test_src
      [F] readme.md
  [D] __pycache__
    [F] main.cpython-311.pyc
    [F] embedding_model.cpython-311.pyc
    [F] filestructure_extraction.cpython-311.pyc
    [F] __init__.cpython-311.pyc
  [F] README.MD
  [F] setup.py
  [D] homebrew 
    [F] filetalk.rb
  [D] notebooks
    [F] meta_data_search.ipynb
  [D] src
    [D] utils
      [F] model_utils.py
      [F] __init__.py
      [D] __pycache__
        [F] embedding_model.cpyth

In [25]:
def find_closest_node(root, query_vector, max_similarity=-np.inf, closest_node=None):
    """
    Finds the node whose README vector is closest to the query vector in terms of cosine similarity.
    """
    if root.readme_vector is not None:
        similarity = np.dot(query_vector, root.readme_vector) / (np.linalg.norm(query_vector) * np.linalg.norm(root.readme_vector))
        if similarity > max_similarity:  # Changed to check for greater similarity
            max_similarity = similarity
            closest_node = root

    for child in root.children:
        closest_node, max_similarity = find_closest_node(child, query_vector, max_similarity, closest_node)

    return closest_node, max_similarity

def search_and_read_readmes(root, query):
    query_vector = embedding_model.encode(query)
    closest_node, _ = find_closest_node(root, query_vector)

    # Read READMEs from the closest node to the root
    current_node = closest_node
    while current_node is not None:
        if current_node.readme_content:
            print(f"folder_path: {os.path.abspath(current_node.path)}")
        current_node = current_node.parent

# Usage
tree_root = build_tree(root_directory)  # Build the tree
search_and_read_readmes(tree_root, "testing")

folder_path: /Users/vijaykatta/Documents/Projects/local_doc_search/tests/test_folder
